In [2]:
import pandas as pd
import numpy as np

In [4]:
filename = "/Users/ahmedabukar/Downloads/:/data/nyc_taxi_2019-07.csv"
df = pd.read_csv(filename,
                usecols=["tpep_pickup_datetime", "tpep_dropoff_datetime",
                        "passenger_count", "trip_distance",
                        "total_amount"],
                parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2019-07-01 00:51:04,2019-07-01 00:51:33,1.0,0.00,4.94
1,2019-07-01 00:46:04,2019-07-01 01:05:46,1.0,4.16,20.30
2,2019-07-01 00:25:09,2019-07-01 01:00:56,1.0,18.80,70.67
3,2019-07-01 00:33:32,2019-07-01 01:15:27,1.0,18.46,66.36
4,2019-07-01 00:00:55,2019-07-01 00:13:05,0.0,1.70,15.30


In [3]:
# create a new column, trip_time, containing the amount of time each taxi ride took as timedelta
df["trip_time"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]

In [4]:
# determine the number and percentage of rides that took less than 1 minute
rides_that_took_1 = df.loc[df["trip_time"] < '1 minute', "trip_time"]
len(rides_that_took_1) / len(df) * 100

1.1126361022936828

In [5]:
# determine the average fare paid by people taking this trips?
(
    df
    .loc[
    df["trip_time"] < "1 minute", 
    "total_amount"]
    .mean()
)

30.397584031219733

In [6]:
# determine the number and percentage of rides that took less than 10 hours?
(
    len(df
    .loc[df["trip_time"] > "10 hours", "trip_time"])
    / len(df)
) * 100

0.2646100045020782

In [7]:
# create a new column, trip_time_group, in which the values are short(<10 minutes),
# medium (≥ between 10 minutes and 1 hour) and long (1 > hour)
df["trip_time_group"] = (
    pd.cut(
        df["trip_time"], 
        bins=[pd.to_timedelta(value) 
              for value in ["0 seconds", "10 minutes", "60 minutes", "100 hours"]],
      labels=["Short", "Medium", "Long"])
)

In [8]:
# determine the propotion of rides in each group
df["trip_time_group"].value_counts(normalize=True) * 100

trip_time_group
Medium    55.323890
Short     43.461438
Long       1.214672
Name: proportion, dtype: float64

In [9]:
# for each value in trip_time_group, determine the average number of passengers.
(
    df.groupby("trip_time_group", observed=True)
    ["passenger_count"]
    .mean()
)

trip_time_group
Short     1.552411
Medium    1.585806
Long      1.700859
Name: passenger_count, dtype: float64

In [10]:
# how many trips are not from july 2019
(
    df
    .query("(tpep_pickup_datetime.dt.year != 2019) and (tpep_pickup_datetime.dt.month != 7)")
    ["tpep_pickup_datetime"]
    .count()
)

56

In [12]:
# what was the mean trip time for each number of passengers?
(
    df
    .groupby("passenger_count")
    ["trip_time"]
    .mean()
    .sort_values()
)

passenger_count
8.0      0 days 00:11:00.500000
0.0   0 days 00:14:18.929810752
7.0   0 days 00:16:38.206896551
1.0   0 days 00:17:46.148103924
2.0   0 days 00:18:34.024342704
3.0   0 days 00:19:02.079604271
4.0   0 days 00:20:10.057290100
6.0   0 days 00:20:54.109564300
5.0   0 days 00:22:29.870464324
9.0      0 days 00:49:16.125000
Name: trip_time, dtype: timedelta64[ns]

In [11]:
# load the taxi data from july 2020 and 2019. for each year and then for each number of passengers
# what was the mean amount paid?
def load_taxi_data(filename):
    return (pd.read_csv(filename,
                          usecols=["tpep_pickup_datetime", "passenger_count", "total_amount"],
                           parse_dates=["tpep_pickup_datetime"])
               .assign(year=lambda df_ : df_["tpep_pickup_datetime"].dt.year)
               )
    
df_july_2020 = load_taxi_data("/Users/ahmedabukar/Downloads/:/data/nyc_taxi_2020-07.csv")

df_july_2019 = load_taxi_data(filename)

(
    pd.concat(
        [df_july_2019, df_july_2020], ignore_index=True)
    .groupby(["year", "passenger_count"])
    ["total_amount"]
    .mean()
)

year  passenger_count
2002  1.0                18.002500
      2.0                18.800000
2008  1.0                18.340000
      2.0                42.860000
      5.0                11.966667
2009  1.0                23.923571
      2.0                45.316000
2010  2.0                18.360000
2019  0.0                18.981793
      1.0                19.284646
      2.0                20.097442
      3.0                20.208111
      4.0                21.063172
      5.0                19.419311
      6.0                19.386516
      7.0                70.080690
      8.0                74.760455
      9.0                93.509375
2020  0.0                16.538912
      1.0                16.856554
      2.0                17.188322
      3.0                17.103106
      4.0                17.964939
      5.0                16.725836
      6.0                16.812911
      7.0                22.456000
      8.0                10.300000
      9.0                11.76000

In [10]:
(
    
)

year  passenger_count
2002  1.0                18.002500
      2.0                18.800000
2008  1.0                18.340000
      2.0                42.860000
      5.0                11.966667
2009  1.0                23.923571
      2.0                45.316000
2010  2.0                18.360000
2019  0.0                18.981793
      1.0                19.284646
      2.0                20.097442
      3.0                20.208111
      4.0                21.063172
      5.0                19.419311
      6.0                19.386516
      7.0                70.080690
      8.0                74.760455
      9.0                93.509375
2020  0.0                16.538912
      1.0                16.856554
      2.0                17.188322
      3.0                17.103106
      4.0                17.964939
      5.0                16.725836
      6.0                16.812911
      7.0                22.456000
      8.0                10.300000
      9.0                11.76000

In [9]:
df_july_2020["passenger_count"]

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
800407    NaN
800408    NaN
800409    NaN
800410    NaN
800411    NaN
Name: passenger_count, Length: 800412, dtype: float64